# The Counted Project
Exploratory analysis with Sankey Diagram about 'People Killed by the Police'

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from matplotlib.sankey import Sankey

In [2]:
# display all the columns
pd.set_option('display.line_width', 5000)
pd.set_option('display.max_columns', 60)
pd.set_option('display.max_rows', 500)

line_width has been deprecated, use display.width instead (currently both are
identical)



In [3]:
dataset = pd.read_csv('/Users/annalie/Downloads/thecounted-data/the-counted-2016.csv')

In [4]:
dataset.head()

,uid,name,age,gender,raceethnicity,month,day,year,streetaddress,city,state,classification,lawenforcementagency,armed
0,20161,Joshua Sisson,30,Male,White,January,1,2016,4200 6th Ave,San Diego,CA,Gunshot,San Diego Police Department,Knife
1,20162,Germonta Wallace,30,Male,Black,January,3,2016,2600 Watson Dr,Charlotte,NC,Gunshot,Charlotte-Mecklenburg Police Department,Firearm
2,20163,Sean O'Brien,37,Male,White,January,2,2016,100 Washington St,Livingston,MT,Gunshot,Livingston Police Department,Knife
3,20164,Rodney Turner,22,Male,Black,January,4,2016,3600 NW 42nd St,Oklahoma City,OK,Gunshot,Oklahoma City Police Department,Firearm
4,20165,Eric Senegal,27,Male,Black,January,4,2016,Gene Stanley Rd,Ragley,LA,Gunshot,Beauregard Parish Sheriff's Office,Unknown


In [5]:
print "number of rows =", len(dataset)
print "number of columns =", len(dataset.columns)

number of rows = 749
number of columns = 14


In [6]:
# check on duplicated rows
dataset = dataset.drop_duplicates()
print "number of rows after removing duplicates = ", len(dataset)

number of rows after removing duplicates =  749


In [7]:
# drop rows with missing values
dataset = dataset.dropna()
print "number of rows after removing missing values =", len(dataset)

number of rows after removing missing values = 732


In [8]:
dataset.armed.unique()

array(['Knife', 'Firearm', 'Unknown', 'No', 'Non-lethal firearm', 'Other',
       'Vehicle', 'Disputed'], dtype=object)

In [9]:
# print number of rows with 'functional', 'non functional' and 'functional needs repair' 
print "Knife =", len(dataset[dataset.armed == 'Knife'])
print "Firearm =", len(dataset[dataset.armed == 'Firearm'])
print "Unkown =", len(dataset[dataset.armed == 'Unkown'])
print "No =", len(dataset[dataset.armed == 'No'])
print "Non-lethal firearm =", len(dataset[dataset.armed == 'Non-lethal firearm'])
print "Other =", len(dataset[dataset.armed == 'Other'])
print "Vehicle =", len(dataset[dataset.armed == 'Vehicle'])
print "Disputed =", len(dataset[dataset.armed == 'Disputed'])

Knife = 115
Firearm = 343
Unkown = 0
No = 111
Non-lethal firearm = 33
Other = 38
Vehicle = 23
Disputed = 5


# Create group by tables

In [10]:
# select variables to use for group by counts
state_raceethnicity_armed = pd.DataFrame(dataset, columns = ['state', 'raceethnicity', 'armed'])
state_raceethnicity_armed.head()

,state,raceethnicity,armed
0,CA,White,Knife
1,NC,Black,Firearm
2,MT,White,Knife
3,OK,Black,Firearm
4,LA,Black,Unknown


In [11]:
# print counts by race ethnicity
print state_raceethnicity_armed.groupby('raceethnicity').size()
    # groupedby_raceethnicity = state_raceethnicity_armed.groupby('raceethnicity').count()
    # print groupedby_raceethnicity
    
# print counts by armed
print state_raceethnicity_armed.groupby('armed').size()
    # groupedby_raceethnicity = state_raceethnicity_armed.groupby('raceethnicity').count()
    # print groupedby_raceethnicity    

# print counts by race ethnicity and type of armed
print state_raceethnicity_armed.groupby(['raceethnicity', 'armed']).size().unstack()
    # or: groupedby_raceethnicity_armed = state_raceethnicity_armed['armed'].groupby([state_raceethnicity_armed['raceethnicity'], state_raceethnicity_armed['armed']]).count()
    # print groupedby_raceethnicity_armed

raceethnicity
Arab-American               2
Asian/Pacific Islander     13
Black                     183
Hispanic/Latino           126
Native American            12
Unknown                    41
White                     355
dtype: int64
armed
Disputed                5
Firearm               343
Knife                 115
No                    111
Non-lethal firearm     33
Other                  38
Unknown                64
Vehicle                23
dtype: int64
armed                   Disputed  Firearm  Knife  No  Non-lethal firearm  Other  Unknown  Vehicle
raceethnicity                                                                                    
Arab-American                NaN        1      1 NaN                 NaN    NaN      NaN      NaN
Asian/Pacific Islander       NaN        4      4   1                   1      1      NaN        2
Black                          4      101     12  31                  12      3       14        6
Hispanic/Latino              NaN       56     

# Prepare data for Sankey Diagram in Tableau

In [12]:
# 1. take a selection of the dataset:
# index 
# ethnicity: black, hispanic, white
# armed: firearm, knife, no, non-lethal firearm, other, unknown, vehicle

# 2. resort this new dataset by the variable armed
# what you get is a reordening of the index numbers
# save only the reordered index variable

# 3. add the reorded index variable to the dataset in step 1

In [17]:
# 1. take a selection of the dataset:
# index 
# ethnicity: black, hispanic, white
# armed: firearm, knife, no, non-lethal firearm, other, unknown, vehicle

dataset_sankey = pd.DataFrame(dataset, columns = ['uid', 'raceethnicity', 'armed'])

# order dataset by race ethnicity
dataset_sankey_sorted_by_race = dataset_sankey.sort('raceethnicity')

# reorder the index from 1 to the end. this will be the new order of the records
# create a new column for the index column
dataset_sankey_sorted_by_race.index = list(range(len(dataset_sankey_sorted_by_race.index)))
dataset_sankey_sorted_by_race['first'] = dataset_sankey_sorted_by_race.index
dataset_sankey_sorted_by_race = pd.DataFrame(dataset_sankey_sorted_by_race, columns = ['first', 'raceethnicity', 'armed'])

dataset_sankey_sorted_by_race.head()

,first,raceethnicity,armed
0,0,Arab-American,Firearm
1,1,Arab-American,Knife
2,2,Asian/Pacific Islander,Firearm
3,3,Asian/Pacific Islander,Knife
4,4,Asian/Pacific Islander,Knife


In [19]:
# 2. resort this new dataset by the variable armed
# what you get is a reordening of the index numbers
# save only the reordered index variable

dataset_sankey_sorted_by_armed = dataset_sankey_sorted_by_race.sort('armed')
# dataset_sankey_sorted = pd.DataFrame(dataset_sankey_sorted, columns = ['random', 'raceethnicity', 'armed'])
dataset_sankey_sorted_by_armed.head(n=10)

,first,raceethnicity,armed
100,100,Black,Disputed
693,693,White,Disputed
140,140,Black,Disputed
103,103,Black,Disputed
45,45,Black,Disputed
582,582,White,Firearm
583,583,White,Firearm
585,585,White,Firearm
586,586,White,Firearm
587,587,White,Firearm


In [20]:
# 3. add the reorded index variable to the dataset in step 1

# reorder the index from 1 to the end. this will be the new order of the records
# create a new column for the index column
dataset_sankey_sorted_by_armed.index = list(range(len(dataset_sankey_sorted_by_armed.index)))
dataset_sankey_sorted_by_armed['second'] = dataset_sankey_sorted_by_armed.index
dataset_sankey_sorted_by_armed.head()

,first,raceethnicity,armed,second
0,100,Black,Disputed,0
1,693,White,Disputed,1
2,140,Black,Disputed,2
3,103,Black,Disputed,3
4,45,Black,Disputed,4


In [21]:
# save dataframe to excel file

dataset_sankey_tableau = dataset_sankey_sorted_by_armed

dataset_sankey_tableau.to_excel('counted_tableau.xls')

# Continue with Tableau

https://public.tableau.com/profile/publish/SankeyDiagramCounteddataset/Dashboard2#!/publish-confirm

In [ ]:
Note: The dataset seems to be duplicated, so the count